# Project 02 - Due Friday, November 11 at 12pm

*Objectives*: Gain experience loading a transactional dataset into a relational database model you define yourself and using SQL to explore its contents. Transform the data into a star schema, documenting the schema visually, and explore the transformed data analytically by writing and executing a number of SQL queries using common syntax and functions and describing your findings.  Gain practice working on these tasks with a partner.

*Grading criteria*: The tasks should all be completed, and questions should all be answered with SQL queries, diagrams, and charts in the space provided, unless a text answer is requested.  The notebook itself should be completely reproducible at datanotebook.org, from start to finish: another person should be able to use the same code to obtain the same results as yours.  Note that you will receive no more than partial credit if you do not add text/markdown cells explaining your thinking at each major step in each problem.

*Attestation*: **Work in pairs**.  At the end of your submitted notebook, identify the work each partner performed and attest that each contributed substantially to the work.

*Deadline*: Friday, November 11, 12pm.  One member of each pair must submit your notebook to Blackboard; you should not both submit it separately.  You may each push a copy to your own GitHub repository.


## Setup - select and obtain data

The US City Open Data Census has a variety of [transactional data from all over the country](http://us-city.census.okfn.org/).  Select one topic and one city from this Census, download the data, and explain your reason for this choice before proceeding.  Provide a link to the web page for the data set you've chosen.

I strongly encourage you to select a dataset from among the following topics:  Crime, Property Assessment, Campaign Finance Contributions, Service Requests (311), and Web Analytics.

By *transactional* data I mean records at the grain of one event per business process.  In the case of Service Requests (311), for example, that would require data at the level of each individual report of a service request.  If this isn't clear, think about the bike trip data - each individual ride was included - and look for data at that level of event/process specificity.  Avoid data like the Connecticut boating data, which was summarized by year, not individual transactions.

Please aim for a dataset of at least 10,000 individual records, but less than 250,000 records.  A little more or a little less is okay, but try to stay within these limits.

## Problem 1 - examine the data (20 points)

Use `csvstat`, `pandas`, or other tools as you see fit and observe the contents of your dataset.  Review important points from any metadata provided.  Describe what you see:  What do the columns mean?  Are there null values?  Which columns interest you the most?  Which columns present some opportunities, challenges, or questions you would like to ask?

<><><

We examine New York City Campaign Contributions in the current 2017 campaign cycle.

Data is available online from the New York City Campaign Finance Board at http://www.nyccfb.info/. The 2017 contributions dataset (in csv format) is available directly via URL link. We begin by retrieving the dataset. All code in this notebook should be entirely replicable if run on datanotebok.org in the environment set up for the GWU course ISTM 6212 Data Management. 

NOTE: This is a happy result. It took many hours to distill. Why? Because the raw csv data file is not compliant with the UTF-8 character set (as you will see) and we are not intimately familiar with the various character sets and tracing down error codes. The csv data file contains at least one instance of a character (0x92) which cannot be decoded by the UTF-8 codec. We specified the ASCII character set and several others and attempted a number of transformations to identify and eliminate the suspect character(s). After a few hours we got lucky with one round of transformations on the dataset (the result was UTF-8 compliant). But it needed to be manually cleaned as additional commas (field delimiters) had been injected into nearly 100 random observations. We cleaned the file and used it as raw data to populate our database, schema and queries. This all took many hours. We prepared to deliver the assignment by attaching the cleaned dataset rather than linking to the source. But we decided to try one more time to see if we could identify a proper codec. Lo & behold, we found that when encoding is specified as 'latin1' the UTF-8 error vanished and the raw contributions dataset could be used directly. So after much consternation, we are able to deliver an elegant solution that gets data from the source, examines and loads it for analysis and summarization.


In [1]:
# First we get the raw data from the NYC Campaign Finance Board Library.

!wget "http://www.nyccfb.info/DataLibrary/2017_Contribution.csv"

--2016-11-11 04:54:25--  http://www.nyccfb.info/DataLibrary/2017_Contribution.csv
Resolving www.nyccfb.info (www.nyccfb.info)... 107.181.16.98
Connecting to www.nyccfb.info (www.nyccfb.info)|107.181.16.98|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4181720 (4.0M) [application/octet-stream]
Saving to: ‘2017_Contribution.csv’

2017_Contribution.c 100%[=====================>]   3.99M  11.2MB/s   in 0.4s   

2016-11-11 04:54:25 (11.2 MB/s) - ‘2017_Contribution.csv’ saved [4181720/4181720]



In [2]:
# The dataset is not UTF-8 compliant and csvclean balks at it as do other simple csvkit commands.

!csvclean 2017_Contribution.csv

Your file is not "utf-8" encoded. Please specify the correct encoding with the -e flag. Use the -v flag to see the complete error.


In [3]:
# A detailed listing showing the problem character (0x92).

!csvclean -v 2017_Contribution.csv

Traceback (most recent call last):
  File "/opt/conda/bin/csvclean", line 6, in <module>
    sys.exit(csvkit.utilities.csvclean.launch_new_instance())
  File "/opt/conda/lib/python3.5/site-packages/csvkit/utilities/csvclean.py", line 76, in launch_new_instance
    utility.main()
  File "/opt/conda/lib/python3.5/site-packages/csvkit/utilities/csvclean.py", line 43, in main
    for row in checker.checked_rows():
  File "/opt/conda/lib/python3.5/site-packages/csvkit/cleanup.py", line 79, in checked_rows
    for row in self.reader:
  File "/opt/conda/lib/python3.5/site-packages/csvkit/py3.py", line 22, in __next__
    return next(self.reader)
  File "/opt/conda/lib/python3.5/site-packages/csvkit/cli.py", line 57, in __next__
    return next(self.f)
  File "/opt/conda/lib/python3.5/codecs.py", line 321, in decode
    (result, consumed) = self._buffer_decode(data, self.errors, final)
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 4570: invalid start byte


In [4]:
# An elegant solution to the anomaly in the raw data file: specify encoding as 'latin1'.

!csvclean -e latin1 2017_Contribution.csv

No errors.


In [5]:
# Total number of rows (includes header row).

!wc -l 2017_Contribution.csv

21239 2017_Contribution.csv


In [6]:
# Listing of all variables.

!csvcut -n 2017_Contribution.csv

  1: ELECTION
  2: OFFICECD
  3: RECIPID
  4: CANCLASS
  5: RECIPNAME
  6: COMMITTEE
  7: FILING
  8: SCHEDULE
  9: PAGENO
 10: SEQUENCENO
 11: REFNO
 12: DATE
 13: REFUNDDATE
 14: NAME
 15: C_CODE
 16: STRNO
 17: STRNAME
 18: APARTMENT
 19: BOROUGHCD
 20: CITY
 21: STATE
 22: ZIP
 23: OCCUPATION
 24: EMPNAME
 25: EMPSTRNO
 26: EMPSTRNAME
 27: EMPCITY
 28: EMPSTATE
 29: AMNT
 30: MATCHAMNT
 31: PREVAMNT
 32: PAY_METHOD
 33: INTERMNO
 34: INTERMNAME
 35: INTSTRNO
 36: INTSTRNM
 37: INTAPTNO
 38: INTCITY
 39: INTST
 40: INTZIP
 41: INTEMPNAME
 42: INTEMPSTNO
 43: INTEMPSTNM
 44: INTEMPCITY
 45: INTEMPST
 46: INTOCCUPA
 47: PURPOSECD
 48: EXEMPTCD
 49: ADJTYPECD
 50: RR_IND
 51: SEG_IND
 52: INT_C_CODE


In [7]:
# Summary statistics for all variables. Key metadata presented after these statistics.

!csvstat -e latin1 2017_Contribution.csv

  1. ELECTION
	<class 'int'>
	Nulls: False
	Values: 2017
  2. OFFICECD
	<class 'int'>
	Nulls: False
	Values: 1, 2, 4, 5, 6
  3. RECIPID
	<class 'str'>
	Nulls: False
	Unique values: 70
	5 most frequent values:
		326:	2126
		148:	1423
		1160:	1360
		264:	1326
		480:	1225
	Max length: 4
  4. CANCLASS
	<class 'str'>
	Nulls: False
	Values: UN
  5. RECIPNAME
	<class 'str'>
	Nulls: False
	Unique values: 70
	5 most frequent values:
		de Blasio, Bill:	2126
		Stringer, Scott:	1423
		Diaz Jr., Ruben:	1360
		Katz, Melinda R:	1326
		James, Letitia:	1225
	Max length: 29
  6. COMMITTEE
	<class 'str'>
	Nulls: False
	Unique values: 9
	5 most frequent values:
		P:	4266
		K:	3324
		N:	2847
		L:	2341
		H:	2265
	Max length: 1
  7. FILING
	<class 'int'>
	Nulls: False
	Values: 1, 2, 3, 4, 5
  8. SCHEDULE
	<class 'str'>
	Nulls: False
	Values: D, N, M, ABC
  9. PAGENO
	<class 'NoneType'>
	Nulls: True
	Values: 
 10. SEQUENCENO
	<class 'NoneType'>
	Nulls: True
	Values: 
 11. REFNO
	<class 'str'>
	Nulls: False
	U

#### Overview of variables

A full key can be found in the accompanying CY2013 CONTRIBUTION KEY spreadsheet.

OFFICECD = Office Code [1=Mayor; 2=Public Advocate; 3=Comptroller; 4=Borough President; 5=City Council; 6=Undeclared; IS=Independent Spender]

RECIPID = Recipient ID shows that 70 candidates are running (contributions as of July 2016)

C_CODE = Contributor Code [CAN=Candidate; CORP=Corporation; EMPO=Labor Union; FAM=Candidate's Family; IND=Individual; LLC=Limited Liability Company; OTHR=Other; PART=Partnership; PCOMC=Candidate Committee; PCOMP=Political Action Committee; PCOMZ=Party Committee; SPO=Candidate's Spouse; UNKN=Unknown]

BOROUGHCD = Contributor's Borough Code: [K=Brooklyn; M=Manhattan; Q=Queens; S=Staten Island; X=Bronx; Z=Outside New York City]

EMPNAME = Contributor's Employer

AMNT = Amount

MATCHAMNT = Matchable Amount of Contribution

INTERMNAME = Intermediary's Name


#### Summary

Races: Mayor, Public Advocate, Borough President (all but Manhattan) and many City Council and undeclared seats are to be contested.

From RECIPID and RECIPNAME it is evident that as of July 2016, Bill deBlasio has received nearly 90% of the total number of individual contributions in the Mayor's race and only two other Mayoral candidates had received any contributions. James Letitia appears to be running uncontested in the race for Public Advocate as he is the only candidate for this office to report contributions. As of July 2016 only four candidates (one for each of the 4 borough presidencies being contested: Staten Island, Brooklyn, Queens and the Bronx) had reported any contributions. So these races also appear to be uncontested.

The central facts for this dataset are AMNT and MATCHAMNT: the dollar amount and matchable amount associated with each contribution. Data is reported at the transaction level. We assume the matchable amount is indeed matched so that the total amount of any contribution is the contribution itself + any matchable amount. 

There are many blank values in the data set but very few, if any, missing values. All critical variables (namely the contributors, recipients and amounts) are fully specified with no missing values. So we should be able to summarize contributions by candidate for each office and identify the shares that come from individuals, candidates or their families, unions, business entities and political committees. 

An initial look at the contributor codes in the raw data shows that the vast majority of official campaign contribution transactions in the data set are from individuals. But the high value contributions from businesses and unions alike generally appear to be routed through political committees (Political Action Committees (PACs) are one type of political committee). Because so much of the big money goes through PACs, the data would need to be processed further to determine the extent of corporate or union backing behind any candidate. Nonetheless, the contributor codes as reported should be rich enough to shed some light on which groups are backing which candidates. We may look only at city-wide and borough-wide offices as the city council and undeclared seats may be too numerous to yield any insight at this level.

For this project we are interested in the recipient (RECIPID and RECIPNAME), office sought (OFFICECD), contributor (NAME and C_CODE) date of transaction (DATE and REFUNDDATE) and the amounts (AMNT and MATCHAMNT). As such we could drop most of the variables in the dataset. Thirty-two of the 52 variables will not be used at this stage. Many of these are entirely or mostly blank or otherwise sparse -- e.g., there was no intermediary in most cases, there is no employer in many cases, etc. But we shall retain all variables since the dataset is already a reasonable size (4MB) and we would not gain a great deal by dropping mostly sparse variables.




## Problem 2 - define a database model, load the data, and explore (20 points)

Based on what you found above, create and connect to a new database, define a database table in it, and load this dataset into it.  You may use either of the methods for this step you have seen in earlier class notebooks.  You may choose to eliminate variables/columns if they are not relevant or interesting to you - explain your reasoning if you do.  Either way, you should load a majority of the columns present in the source dataset as it is in its raw form into the database, and all of its rows.

Once your data has loaded successfully, run a `COUNT(*)` query to verify that all the data has loaded correctly.

Explore the data to zero in on a few themes you would like to further study analytically.  Discuss columns that present opportunities for extraction into dimensions, and identify the specific columns that contain facts you want to measure.

In [8]:
!echo 'redspot' | sudo -S service postgresql restart

[sudo] password for jovyan: Restarting PostgreSQL 9.5 database server: main.


In [9]:
!createdb -U dbuser NYC_CampgnFin

In [10]:
%load_ext sql

/opt/conda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/opt/conda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [11]:
%sql postgresql://dbuser@localhost:5432/NYC_CampgnFin

'Connected: dbuser@NYC_CampgnFin'

In [12]:
%%sql
DROP TABLE IF EXISTS Master;
CREATE TABLE Master (
    election_year INTEGER,
    office_cd INTEGER,
    recip_id VARCHAR(4),
    can_class CHAR(2),
    recip_name CHAR(30),
    committee CHAR(1),
    filing INTEGER,
    schedule CHAR(3),
    page_no CHAR(1),
    seq_no CHAR(1),
    ref_no VARCHAR(8),
    date TIMESTAMP,
    refund_date TIMESTAMP,
    donor_name CHAR(35),
    contr_cd CHAR(6),
    str_no CHAR(1),
    str_name CHAR(1),
    apt CHAR(1),
    borough CHAR(4),
    city CHAR(22),
    state CHAR(2),
    zip CHAR(5),
    occupation CHAR(30),
    emp_name CHAR(30),
    emp_str_no CHAR(25),
    emp_str_name CHAR(32),
    emp_city CHAR(22),
    emp_state CHAR(2),
    amount REAL,
    match_amnt REAL,
    prev_amnt REAL,
    pay_method INTEGER,
    int_erm_no INTEGER,
    int_erm_name CHAR(25),
    int_str_no CHAR(1),
    int_str_name CHAR(1),
    int_apt_no CHAR(1),
    int_city CHAR(16),
    int_state CHAR(2),
    int_zip CHAR(5),
    int_emp_name CHAR(30),
    int_emp_st_no CHAR(6),
    int_emp_st_nm CHAR(25),
    int_emp_city CHAR(16),
    int_emp_st CHAR(2),
    int_occup CHAR(30),
    purpose_cd CHAR(6),
    exempt_cd CHAR(1),
    adj_type_cd CHAR(1),
    rr_ind BOOLEAN,
    seg_ind BOOLEAN,
    int_c_cd CHAR(5)
);

Done.
Done.


[]

In [13]:
!pwd
!ls

/home/jovyan/work
2017_Contribution.csv	   project-02-chen-fitz-take3a.ipynb
2017_Contribution_out.csv


<><><

Notice that we must specify 'latin1' encoding below, else the load will fail.

Apparently, the COPY (table) FROM statement below requires an absolute path, so we provide it ...

Make sure the path below is 'your working directory (from above) /2017_Contribution.csv'

In [16]:
%%sql
COPY Master FROM '/home/jovyan/work/2017_Contribution.csv'
CSV
ENCODING 'latin1'
HEADER
QUOTE '"'
DELIMITER ',';

21238 rows affected.


[]

In [32]:
%%sql
SELECT COUNT (*), SUM(amount), SUM(match_amnt)
FROM Master;

1 rows affected.


count,sum,sum_1
21238,11682900.0,1257150.0


Success. All observations present & accounted for yielding $11.7m & $1.3m in contributions and matching contributions.

The facts we wish to measure are total campaign contributions (amount + match_amnt). 

As an initial exploratory query, it may be illuminating to compare de Blasio's war chest to that of his two rivals in the mayoral race ...just to see if they are even in the same order of magnitude. 


In [48]:
%%sql
SELECT office_cd, recip_name, SUM(amount + match_amnt)
FROM Master
WHERE office_cd = 1
GROUP by office_cd, recip_name
ORDER by recip_name;

3 rows affected.


office_cd,recip_name,sum
1,"Faulkner, Michel J",39770.0
1,"Thompson, Josh",30450.7
1,"de Blasio, Bill",2408790.0


Now `that` is interesting. de Blasio raised nearly 2 orders of magnitude (100x) more than each of his rivals. That's no contest.

As far as dimension tables go, it would be interesting to compare and contrast total contributions in the mayoral race to the races for borough president and all other local seats. Similarly it would be interesting to compare total amounts from individuals to amounts from businesses, unions and political committees. Similarly, it might be instructive to plot contributions over time. 

As such, we may wish to extract recipient and office information into one dimension table, donor information into another and (transaction) date information into another dimension table.



## Problem 3 - define a star schema, and load your data into it (30 points)

Define a star schema consisting of at least one fact table and at least two dimensions.  Transform your source data into the new fact table and dimensions.  Discuss your key modeling decisions.

Document your star schema with a diagram.  There several tools available for this, such as Google Docs and Microsoft Visio.

Star Schema Diagram:  See the separate file.
<img src="files/schema.jpg">

##### Potential Dimension Table:  trans_dimension
Due to both the lack of usefulness in term of generating insights and an implementational difficulty related to the date fields, we later decided to abandon this dimension table. We decided to include the table-creation steps here, however, to capture our thought process. 

In [49]:
%%sql
DROP TABLE IF EXISTS trans_dimension;
CREATE TABLE trans_dimension (
    election_year INTEGER,
    date TIMESTAMP,
    refund_date TIMESTAMP,
    ref_no VARCHAR(8),
    trans_key SERIAL
);

Done.
Done.


[]

In [50]:
%%sql
ALTER TABLE trans_dimension ADD PRIMARY KEY (trans_key);

Done.


[]

In [51]:
%%sql
INSERT INTO trans_dimension
SELECT DISTINCT election_year, date, refund_date
FROM Master;

1179 rows affected.


[]

Quick validation:

In [52]:
%%sql
SELECT *
FROM trans_dimension
LIMIT 50;

100 rows affected.


election_year,date,refund_date,ref_no,trans_key
2017,2014-06-08 00:00:00,2015-07-11 00:00:00,None,1
2017,2016-03-29 00:00:00,2016-07-06 00:00:00,None,2
2017,2015-12-16 00:00:00,None,None,3
2017,2016-06-14 00:00:00,None,None,4
2017,2014-06-09 00:00:00,None,None,5
2017,2015-03-17 00:00:00,None,None,6
2017,2016-06-17 00:00:00,None,None,7
2017,2015-02-21 00:00:00,None,None,8
2017,2015-02-22 00:00:00,None,None,9
2017,2015-07-11 00:00:00,2015-08-13 00:00:00,None,10


##### Dimension Table 1:  donor_dimension 

In [53]:
%%sql
DROP TABLE IF EXISTS donor_dimension;
CREATE TABLE donor_dimension (
    donor_name CHAR(35),
    borough CHAR(4),
    city CHAR(22),
    state CHAR(13),
    zip CHAR(5),
    contr_cd CHAR(6),
    occupation CHAR(30),
    emp_name CHAR(30),
    donor_key SERIAL
);

Done.
Done.


[]

In [54]:
%%sql
ALTER TABLE donor_dimension ADD PRIMARY KEY (donor_key);

Done.


[]

In [55]:
%%sql
INSERT INTO donor_dimension
SELECT DISTINCT donor_name, borough, city, state, zip, contr_cd, occupation, emp_name
FROM Master;

17624 rows affected.


[]

Quick validation:

In [56]:
%%sql
SELECT *
FROM donor_dimension
LIMIT 50;

100 rows affected.


donor_name,borough,city,state,zip,occupation,emp_name,donor_key
"Tempesta, Donna M",Z,Bethpage,NY,11714,CPA,AHF,1
"Siddique, Mahammad A",Q,Jamaica,NY,11433,Hospotality,Starwood Hotel and Restaurant,2
"Nicolich, Sergio",Q,Flushing,NY,11358,Police Officer,Nassau County,3
"Keren, Donna",M,New York,NY,10004,Researcher,NYC & Company,4
"Carr, Dwaine",Z,Roselle,nj,7203,None,None,5
"Mehas, Carol",M,New York,NY,10011,Homemaker,None,6
"Dunston, Jeffrey",K,Brooklyn,NY,11230,CEO,Nebhco,7
"Berman, Carolyn v",K,Brooklyn,NY,11217,Associate Professor,New School,8
"Carr, Michael",S,Staten Island,NY,10304,retired,None,9
"Drenis, Jerry",Z,Port Washington,NY,11050,CEO,Marathon Energy,10


##### Dimension Table 2:  recipient_dimension

In [57]:
%%sql
DROP TABLE IF EXISTS recipient_dimension;
CREATE TABLE recipient_dimension (
    recip_id VARCHAR(4),
    recip_name CHAR(30),
    committee CHAR(1),
    office_cd INTEGER  
);

Done.
Done.


[]

In [58]:
%%sql
ALTER TABLE recipient_dimension ADD PRIMARY KEY (recip_id);

Done.


[]

In [59]:
%%sql
INSERT INTO recipient_dimension
SELECT DISTINCT recip_id, recip_name, committee, office_cd 
FROM Master;

70 rows affected.


[]

Quick validation:

In [60]:
%%sql
SELECT *
FROM recipient_dimension;

70 rows affected.


recip_id,recip_name,committee,office_cd
1719,"Treyger, Mark",J,5
1627,"Cohen, Andrew J",J,5
1589,"Cumbo, Laurie A",J,5
337,"Van Bramer, James G",L,6
353,"Rodriguez, Ydanis A",M,6
1707,"Gibson, Vanessa L",K,5
1673,"Espinal, Jr., Rafael L",K,6
1928,"Marano, John C",H,5
1694,"Menchaca, Carlos",J,5
148,"Stringer, Scott",P,6


##### Fact Table:  campgn_fact

In [61]:
%%sql
DROP TABLE IF EXISTS campgn_fact;
CREATE TABLE campgn_fact (
    recip_id VARCHAR(4),
    date TIMESTAMP,
    refund_date TIMESTAMP,
    donor_name CHAR(35),
    recip_name CHAR(30),
    ref_no VARCHAR(8),
    amount REAL,
    match_amnt REAL,
    prev_amnt REAL,
    total_contribution REAL
);

Done.
Done.


[]

In [62]:
%%sql
INSERT INTO campgn_fact
SELECT recip_id, date, refund_date, donor_name, recip_name, ref_no, amount, match_amnt, prev_amnt
FROM Master;

21238 rows affected.


[]

New column: The total amount of contribution should be the sum of individual donation and company-matched donation.

In [63]:
%%sql
UPDATE campgn_fact
SET total_contribution = amount + match_amnt;

21238 rows affected.


[]

In [64]:
%%sql
ALTER TABLE campgn_fact ADD COLUMN donor_key INTEGER;

Done.


[]

In [65]:
%%sql
UPDATE campgn_fact
SET donor_key = donor_dimension.donor_key
FROM donor_dimension
WHERE campgn_fact.donor_name = donor_dimension.donor_name;

21238 rows affected.


[]

Note: As discussed earlier, trans_dimension table had been removed from our star schema:

In [ ]:
#%%sql
#ALTER TABLE campgn_fact ADD COLUMN trans_key VARCHAR(10);

In [ ]:
#%%sql
#UPDATE campgn_fact
#SET trans_key = trans_dimension.trans_key
#FROM trans_dimension
#WHERE campgn_fact.date = trans_dimension.date
#AND campgn_fact.refund_date = trans_dimension.refund_date
#AND campgn_fact.ref_no = trans_dimension.ref_no;

In [ ]:
%%sql
ALTER TABLE campgn_fact ADD PRIMARY KEY (donor_key,recip_id,ref_no);

In [66]:
%%sql
ALTER TABLE campgn_fact DROP COLUMN date, DROP COLUMN refund_date, DROP COLUMN donor_name, DROP COLUMN recip_name;

Done.


[]

Quick validation:

In [67]:
%%sql
SELECT COUNT(*), SUM(total_contribution)
FROM campgn_fact;

1 rows affected.


sum
12940100.0


In [69]:
%%sql
SELECT *
FROM campgn_fact
LIMIT 50;

100 rows affected.


recip_id,amount,match_amnt,prev_amnt,total_contribution,donor_key
1938,50.0,0.0,0.0,50.0,7770
1710,2000.0,0.0,0.0,2000.0,5724
530,2750.0,0.0,0.0,2750.0,5724
1559,2750.0,0.0,0.0,2750.0,16750
1176,2500.0,0.0,0.0,2500.0,13508
1164,2750.0,0.0,0.0,2750.0,9273
264,3850.0,0.0,0.0,3850.0,14448
1335,2750.0,0.0,0.0,2750.0,1316
1160,3850.0,0.0,0.0,3850.0,9994
148,4950.0,0.0,0.0,4950.0,13753


## Problem 4 - explore your data in its new schema (30 points)

Strictly using the dimensional model tables you defined and populated for Problem 3, explore your data in its new form.  Use any of the query strategies we've seen in class, including transformations, aggregates, subqueries, rollups, and, of course, joins.  Add plots to highlight particular themes that stand out.  Describe your thinking and observations along the way.

In [ ]:
%%sql
SELECT contr_cd, SUM(total_contribution) AS total, COUNT(contr_cd) AS count
FROM donor_dimension AS a, campgn_fact AS b
WHERE a.donor_key=b.donor_key
GROUP BY contr_cd
ORDER BY total DESC;

##### Query Result 1: Top Types of Contributors
The top 3 are individual, political action committee, & labor union

In [ ]:
%matplotlib inline

In [ ]:
%%sql
SELECT office_cd, SUM(total_contribution) AS contribution
FROM recipient_dimension AS a, campgn_fact AS b
WHERE a.recip_id=b.recip_id
GROUP BY office_cd
ORDER BY contribution DESC;

In [ ]:
result = _
result.bar()

##### Query Result 2: Top Offices Receiving Contributions
The top 5 are Undecided, City Council, Mayor, Borough President, & Public Advocate.  What does "Undecided" mean?  Is this money laundering?  We need further background information and do a deeper analysis.

In [ ]:
%%sql
SELECT donor_name, contr_cd, recip_name, office_cd, SUM(total_contribution) AS contribution
FROM donor_dimension AS a, recipient_dimension AS b, campgn_fact AS c
WHERE a.donor_key=c.donor_key
AND b.recip_id=c.recip_id
GROUP BY donor_name, contr_cd, recip_name, office_cd
ORDER BY contribution DESC
LIMIT 10;

##### Query Result 3: Top 10 Donors
1.  9 out of 10 were individuals.
2.  4 out of 10 went to the mayoral candidate Bill de Blasio.
3.  Martha donated about 8,000 dollars to herself.
4.  About 0.6% (81.85k / 12.9mm) of the overall finance contribution came from the top 10.

In [ ]:
%%sql
SELECT emp_name, SUM(amount) AS original, SUM(match_amnt) AS matching, SUM(total_contribution) AS total 
FROM donor_dimension AS a, campgn_fact AS b
WHERE a.donor_key=b.donor_key
AND emp_name NOT LIKE ('None%')
AND emp_name NOT LIKE ('Self%')
AND emp_name NOT LIKE ('self%')
AND emp_name NOT LIKE ('Retired%')
AND emp_name NOT LIKE ('RETIRED%')
AND emp_name NOT LIKE ('retired%')
AND emp_name NOT LIKE ('Unemployed%')
AND emp_name NOT LIKE ('Not %')
AND emp_name NOT LIKE ('N/A%')
AND emp_name NOT LIKE ('n/a%')
GROUP BY emp_name
ORDER BY matching DESC
LIMIT 20;

##### Query Result 4: Top 20 Matching Employers
No big banks or financial institutions in the top 20?  We would like to look further into Silverstein Properties and Meridian Capital.

## Bonus (10 points for one of A or B)


### Option A - Automating ETL

Consider the work you did to load your original raw dataset and then transform it into a dimensional model for analysis.  What would it take to automate this process?  How often would you need to update the data?  Could you easily automate any data cleaning steps?  What checks would you need to put in place to ensure quality?  Would humans need to be involved, or could you automate it all?

Discuss.


### Option B - Augmenting dimensions

Considering your dimensional model, what external data could you find to augment it?  As in the case of bike trips, where weather might provide an interesting added dimension/context not present in the original data, you can probably find another source of data to complement your own model.  Identify one such source and add it to your model, demonstrating its value with a few new queries.

#### Option A - Automating ETL

We did considerable work up front just to extract usable data from the raw dataset. Our difficulties were due to a bad bit somewhere in the file, a non-standard encoding or some other artifact that the notebook could not handle. Unfortunately `csvclean` was of no use. In a case like this it is hard to see how the ETL process could be automated satisfactorily. 

However, ultimately we did find a solution. Once the raw dataset is consistently readable many of the steps needed to transform it into a dimensional model could be automated, at least in part.

If the dataset is simply to be updated (that is, only the data will change but variable names and value labels will not) then the data update process could be entirely automatic. The data would need to be updated on a regular basis (usually monthly) and would entail running all code in a notebook such as this after the databases are refreshed.

Given our experience with this raw dataset (the UTF-8 non-conformance error in particular) it would take a fairly advanced pattern recognition algorithm and advanced coding to clean a new data set (to do what we did when the raw data failed to load). Moreover, the data cleaning tools at the ready would need to be a whole lot more sophisticated than csvclean. Data cleaning is essentially making sense out of chaos. IBM Watson might approach the task. But anything less adept would likely be unsatisfactory. Some data cleaning steps can always be automated. In fact, a common approach to reducing data entry errors is to place limits on valid values for all new data. This kind of screening technique could easily be employed to reduce the introduction of erroneous data.

Design of the database model is not a deterministic enterprise. There is some art involved. So if the machine is to design the database from the outset, it would need to be equipped with substantial artificial intelligence routines that rely on expert rules for the spectrum of data structures that might (and might not) be expected.

Checksums and hash values, such as the internal sums and counts employed here to ensure all observations have loaded, can be used to check consistency during a data cleaning process. But with the exception of the simple update, humans would need to be involved when internal checksums fail.

#####Conclusion:
Both Daniel and Kevin imported and examined the raw data.  Kevin took the lead in dealing with cleaning issues.  Daniel defined the database model while Kevin explored the data.  Daniel defined the star schema and loaded data into it.  Daniel ran the queries in its new schema.  Kevin wrote the automated ETL for the bonus points.  We each contributed a great deal to this project.